In [ ]:
!pip install transformers datasets torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
from datasets import Dataset, load_dataset
import pandas as pd
from sklearn.model_selection import train_test_split


In [ ]:
# Load and prepare your dataset
df = pd.read_excel("/content/data.xlsx")

df['inputs'] = df['scenario'].fillna("").astype(str) + " [SEP] " + df['witnesses'].fillna("").astype(str)
df['targets'] = df['judgment'].fillna("").astype(str)

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
print(train_df.keys())
print(test_df.keys())

# Convert to Hugging Face datasets
train_dataset = Dataset.from_pandas(train_df[['inputs', 'targets']])
test_dataset = Dataset.from_pandas(test_df[['inputs', 'targets']])


Index(['scenario', 'witnesses', 'judgment', 'inputs', 'targets'], dtype='object')
Index(['scenario', 'witnesses', 'judgment', 'inputs', 'targets'], dtype='object')


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load the pretrained model
model = AutoModelForCausalLM.from_pretrained('facebook/opt-125m')

# Load the corresponding tokenizer
tokenizer = AutoTokenizer.from_pretrained('facebook/opt-125m')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/651 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/251M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

In [ ]:
def tokenize_function(examples):
    # Encode the inputs and prepare labels for a causal language model
    model_inputs = tokenizer(examples['inputs'], truncation=True, padding="max_length", max_length=1024, return_tensors="pt")
    # Labels should be the same as input_ids for language modeling
    model_inputs['labels'] = model_inputs.input_ids.detach().clone()
    return model_inputs

train_dataset = train_dataset.map(tokenize_function, batched=True, remove_columns=train_dataset.column_names)
test_dataset = test_dataset.map(tokenize_function, batched=True, remove_columns=test_dataset.column_names)

# do the traintest split


Map:   0%|          | 0/801 [00:00<?, ? examples/s]

Map:   0%|          | 0/201 [00:00<?, ? examples/s]

In [ ]:
!pip install rouge-score


  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=dafecc0c8adebf61c59ce82ddbe2b55bfbf608c9ef07fc1d42c8c2693583bc5c
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [ ]:
from rouge_score import rouge_scorer

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # Decode predictions and labels
    decoded_preds = [tokenizer.decode(p, skip_special_tokens=True, clean_up_tokenization_spaces=True) for p in predictions]
    decoded_labels = [tokenizer.decode(l, skip_special_tokens=True, clean_up_tokenization_spaces=True) for l in labels]

    # ROUGE scoring
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = {'rouge1': 0, 'rouge2': 0, 'rougeL': 0}

    for pred, label in zip(decoded_preds, decoded_labels):
        score = scorer.score(label, pred)
        scores['rouge1'] += score['rouge1'].fmeasure
        scores['rouge2'] += score['rouge2'].fmeasure
        scores['rougeL'] += score['rougeL'].fmeasure

    # Average the scores
    num_eval_examples = len(decoded_preds)
    scores['rouge1'] /= num_eval_examples
    scores['rouge2'] /= num_eval_examples
    scores['rougeL'] /= num_eval_examples

    return scores


In [ ]:
from transformers import TrainingArguments, Trainer



training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=3,  # Adjust based on your GPU capacity
    num_train_epochs=10,
    weight_decay=0.01
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
)
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-7-8e481e197b83>:13: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,No log,0.500095
2,0.741700,0.481260
3,0.741700,0.482176
4,0.334600,0.489882
5,0.334600,0.504352
6,0.250400,0.521330
7,0.250400,0.535531
8,0.191200,0.551676
9,0.191200,0.561711
10,0.154600,0.567143


TrainOutput(global_step=2670, training_loss=0.3222375855463721, metrics={'train_runtime': 2941.6408, 'train_samples_per_second': 2.723, 'train_steps_per_second': 0.908, 'total_flos': 4185898352640000.0, 'train_loss': 0.3222375855463721, 'epoch': 10.0})

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
model_path = '/content/drive/My Drive/ModelResults/facebook-opt125M'
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)


('/content/drive/My Drive/ModelResults/facebook-opt125M/tokenizer_config.json',
 '/content/drive/My Drive/ModelResults/facebook-opt125M/special_tokens_map.json',
 '/content/drive/My Drive/ModelResults/facebook-opt125M/vocab.json',
 '/content/drive/My Drive/ModelResults/facebook-opt125M/merges.txt',
 '/content/drive/My Drive/ModelResults/facebook-opt125M/added_tokens.json',
 '/content/drive/My Drive/ModelResults/facebook-opt125M/tokenizer.json')

In [ ]:
!pip install openpyxl
!pip install transformers datasets torch evaluate openpyxl
!pip install datasets
!pip install rouge-score datasets
!pip install evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.5 MB/s eta 0:00:00


In [ ]:
import torch
model_path = '/content/drive/My Drive/ModelResults/facebook-opt125M'
model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Move model to the correct device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


OPTForCausalLM(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 768, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 768)
      (final_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (layers): ModuleList(
        (0-11): 12 x OPTDecoderLayer(
          (self_attn): OPTSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,)

In [ ]:
import evaluate
rouge = evaluate.load("rouge")


In [ ]:
import pandas as pd

def clean_text(text):
    # Check if text is not NaN and is a string; otherwise, return it as is or convert to string
    if pd.notna(text) and isinstance(text, str):
        # Regex to match all characters allowed in Excel, which excludes most ASCII control characters
        return re.sub(r'[\x00-\x1f\x7f-\x9f]', '', text)
    return ""


In [ ]:
import pandas as pd
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import evaluate

# Load trained model and tokenizer
model_path = '/content/drive/My Drive/ModelResults/facebook-opt125M'
model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Set up the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Initialize ROUGE for evaluation
rouge = evaluate.load("rouge")

# Define a function to generate predictions
def predict_judgment(input_text):
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True).to(device)
    summary_ids = model.generate(
        inputs["input_ids"],
        max_length=700,
        num_beams=5,
        no_repeat_ngram_size=3,
        length_penalty=1.2,
        early_stopping=True
    )
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Prepare to store results
results = []

# Loop through the test dataset and generate predictions
for index, row in test_df.iterrows():
    scenario = row["scenario"]
    witnesses = row["witnesses"]
    reference_judgment = row["judgment"]
    input_text = f"Scenario: {scenario} Witnesses: {witnesses}"

    # Generate prediction
    predicted_judgment = predict_judgment(input_text)

    # Add the prediction and reference for ROUGE evaluation
    rouge.add(prediction=predicted_judgment, reference=reference_judgment)

    # Collect results for analysis
    results.append({
        "Scenario": scenario,
        "Witnesses": witnesses,
        "Reference Judgment": reference_judgment,
        "Predicted Judgment": predicted_judgment
    })

# Compute ROUGE scores
rouge_scores = rouge.compute()
print("ROUGE Scores:", rouge_scores)

results_df = pd.DataFrame(results)

# # Apply the clean_text function after ensuring all entries are strings
# results_df['Scenario'] = results_df['Scenario'].astype(str).apply(clean_text)
# results_df['Witnesses'] = results_df['Witnesses'].astype(str).apply(clean_text)
# results_df['Reference Judgment'] = results_df['Reference Judgment'].astype(str).apply(clean_text)
# results_df['Predicted Judgment'] = results_df['Predicted Judgment'].astype(str).apply(clean_text)

# Now you can safely save to a CSV file without encountering the TypeError
results_df.to_csv("predicted_judgments_with_rouge.csv", index=False)
print("Predictions and ROUGE scores saved to predicted_judgments_with_rouge.csv")





ROUGE Scores: {'rouge1': 0.3155285872628949, 'rouge2': 0.10293991073222926, 'rougeL': 0.17576194735135475, 'rougeLsum': 0.17792994971460796}
Predictions and ROUGE scores saved to predicted_judgments_with_rouge.csv
